# Mass matrix selection criterion

In [1]:
from sklearn import covariance
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import linalg, stats

In [2]:
np.random.seed(42)

In [3]:
N = 10_000
M = 200

# Generate training and test samples.
# The true eigenvalues are (n - 1) times 1 and 1 times `scale ** 2`

def transform(samples, direction, scale):
    samples_ = samples.copy()
    samples_[...] += (samples @ direction)[:, None] * (direction * (scale - 1))
    return samples_

direction = np.random.randn(N)
direction[...] /= linalg.norm(direction)

scale = 2

train, test = np.random.randn(2, M, N)
train = transform(train, direction, scale)
test = transform(test, direction, scale)

We now assume that we were able to extract the direction and scale exactly from train.

So we reverse-transform our test samples:

In [4]:
test_trafo = transform(test, direction, 1 / scale)

We now test three different measures for how good our transformation is.

### Ledoit-Wolf based max_min eigenvalue selection criterion

In [5]:
test_cov_lw, _ = covariance.ledoit_wolf(test)
test_trafo_cov_lw, _ = covariance.ledoit_wolf(test_trafo)

In [6]:
pre_trafo = np.sqrt(linalg.eigvalsh(test_cov_lw)[-1])
post_trafo = np.sqrt(linalg.eigvalsh(test_trafo_cov_lw)[-1])
pre_trafo, post_trafo, pre_trafo / post_trafo

(1.285992620208809, 1.2851403862853168, 1.0006631446125163)

It reports a very small improvement, but nowhere near the actual value (2).

The absolute estimate is wrong, the true values are 2 and 1.

### Empirical covariance based max eigenvalue criterion

This is pretty much the one from the paper, assuming that the minimum eigenvalue is the same for both.

In [7]:
test_cov_eigs = linalg.svdvals(test) ** 2 / M
test_trafo_eigs = linalg.svdvals(test_trafo) ** 2 / M

In [8]:
pre_trafo = np.sqrt(test_cov_eigs[0])
post_trafo = np.sqrt(test_trafo_eigs[0])
pre_trafo, post_trafo, pre_trafo / post_trafo

(8.064307738438346, 8.060219778530083, 1.0005071772260048)

This also reports a very small improvement, but again not the true value.

The absolute values are *way* off.

### Variance estimate in proposed direction

In [9]:
np.sqrt(direction @ test.T @ test @ direction / M)

2.130233813640912

This correctly tells us that the variance along the direction that was proposed to transform was ~2 as expected.